## Jak nauczyć się sieci rekurencyjnych
- przeczytać 3 razy http://r2rt.com/written-memories-understanding-deriving-and-extending-the-lstm.html
- kolorowe obrazki są tu http://colah.github.io/posts/2015-08-Understanding-LSTMs/

## Restricted Isometry Property (compressed sensing)

https://en.wikipedia.org/wiki/Restricted_isometry_property
http://www.math.ucla.edu/~tao/preprints/sparse.html
https://class.ee.washington.edu/546/2010spr/papers/Baraniuk-RIP-JL.pdf
https://en.wikipedia.org/wiki/Compressed_sensing
http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf

Losowa macierz jest prawie izometrią:
- bierzemy macierz n x m,
- każdy element losujemy z rozkładu o średniej zero i wariancji 1/n,
- rozkład może być dowolny - Glorot bierze rozkład jednostajny na przedziale, któryś z powyższych linków sugeruje, że rozkłady Gaussa dają najlepsze przybliżenie izometrii,
- każda kolumna ma długość średnio jeden, iloczyn skalarny dwóch kolumn jest średnio zero,
- oczywiście im m większe od n, tym gorzej jest upakować w R^n m wektorów wzajemnie prawie prostopadłych, ale podobno to dość dobrze działa,
- compressed sensing - podobno jeśli dane są sparse (ale nie wiemy, wzdłuż jakich kierunków), to przepuszczenie ich przez losową projekcję traci bardzo mało informacji
- dygresja - Glorot chce, aby gradienty przy backpropagation też były potraktowane izometrią, dlatego chciałby też, żeby wiersze były prawie prostopadłe (dlaczego? zrozumieć wzory); w związku z tym chciałby jednocześnie wariancję 1/n i 1/m, stąd n+m w mianowniku jako, wg niego, "kompromis".


## Jak uczyć sieć rekurencyjną

Backpropagation + np. adam, ale...


### Sieć rekurencyjna vs zwykła
http://r2rt.com/written-memories-understanding-deriving-and-extending-the-lstm.html#backpropagation-through-time-and-vanishing-sensitivity
szczególnie warto zrozumieć ten kawałek wpisu na blogu

### Truncated Backpropagation
http://r2rt.com/styles-of-truncated-backpropagation.html

"Suppose we are training an RNN on sequences of length 10,000. If we apply non-truncated backpropagation through time, the entire sequence is fed into the network at once, the error at time step 10,000 will be back propagated all the way back to time step 1. The two problems with this are that it is (1) expensive to backpropagate the error so many steps, and (2) due to vanishing gradients, backpropagated errors get smaller and smaller layer by layer, which makes further backpropagation insignificant."

"Truncated backpropagation processes the sequence one timestep at a time, and every k1 timesteps, it runs BPTT for k2 timesteps."

Dwa rodzaje truncated backpropagation:
- k1 == k2
- k1 == 1

### Real-Time Recurrent Learning
- propagowanie gradientu w przyszłość?
- algorytm dobry do uczenia online?
- jak to działa? czytamy:
    - http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.52.9724&rep=rep1&type=pdf
    - https://www.willamette.edu/~gorr/classes/cs449/rtrl.html
    - http://www.dlsi.ua.es/~mlf/nnafmc/pbook/node29.html
    - https://web.stanford.edu/class/psych209a/ReadingsByDate/02_25/Williams%20Zipser95RecNets.pdf (s. 453)
    - http://www.sciencedirect.com/science/article/pii/S0925231298000897


## Stateful (w sensie kerasa) LSTM

http://philipperemy.github.io/keras-stateful-lstm/
https://keras.io/layers/recurrent/
https://github.com/fchollet/keras/issues/98

- LSTM ma "pamięć" (c, h)
- jak inicjalizujemy pamięć w pierwszym kroku czasowym? (zerami, losowo itp.)
- stateful - pamięć na końcu i-tego przykładu w batchu jest pamięcią na początku i-tego przykładu w następnym batchu

po co?:
- np. chcemy pociąć długie szeregi na batche i umożliwić kerasowi zapamiętanie stanu
- coś jeszcze?

## 10k różnych LSTMów

http://jmlr.org/proceedings/papers/v37/jozefowicz15.pdf

- główne pytanie - dlaczego LSTM/GRU tak, a nie inaczej?
- jakimś algorytmem genetycznym wygenerowali 10000 architektur LSTM (mutacje w grafie obliczeń)
- po wstępnym odfiltrowaniu zostawili 1000
- każdą przetestowali na 230 zestawach hiperparametrów
- wnioski:
    - jak wyszło coś działającego, to raczej było podobne do LSTM/GRU, więc nic bardzo odmiennego od wymyślonych "intuicyjnie" architektur nie znaleźli
    - jak się doda bias == 1 do forget gate w LSTM, to osiąga skuteczność GRU

## Co rysować podczas trenowania

cs231n.github.io/neural-networks-3

### TensorBoard
Jeśli ktoś klepie w TensorFlow (ktoś klepie? pewnie nikt), to może użyć TensorBoard:
https://www.tensorflow.org/versions/r0.11/tutorials/monitors/

### Keras
Przede wszystkim callbacks + własne metryki (keras reportuje nie tylko objective, ale też wybrane metryki)

### "Oczywiste"
- train/valid/test loss/inne_metryki
- może zmiany learning rate?

### Wartości na warstwach
http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf
Glorot bada saturację warstw aktywacji, obserwując ich wartości:
- mean/std outputów z warstw aktywacji - np. jeśli średnia po sigmoidzie jest w okolicy 0 lub 1, to źle,
- "Activation values normalized histogram at the end of learning, averaged across units of the same layer and across 300 test examples",
- może warto połączyć dwa powyższe w wykres 3D? czas (numer epoki) x histogramy?

### Gradient
Interesują nas gradienty dla każdego parametru w każdej warstwie. W wypadku sieci rekurencyjnych chcemy też wiedzieć, jak gradient propaguje się wstecz w czasie.
https://github.com/fchollet/keras/issues/2226
Jak to zrobić? W kerasie mogą być problemy, w tensorflow podobno jest łatwiej - TODO dowiedzieć się.

Obrazek z drugiego paperu o Batch Normalization w sieciach rekurencyjnych:
<img src='img/sieci_rekurencyjne/gradient_backprop_through_time.png' width = 800px>

http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf

Glorot robi podobne rzeczy, co powyżej dla wartości. W szczególności konkluduje, że bez jego inicjalizacji gradienty na niższych warstwach mają dużą wariancję; z jego inicjalizacją wariancje są konsystentne pomiędzy warstwami.

#### Gradient checking
https://www.tensorflow.org/api_docs/python/test/gradient_checking
Można sobie też od czasu do czasu wyestymować gradienty z wzoru f(x + delta) - f(x - delta) / 2 delta

### Delta wartości wag
Na cs231n piszą, żeby patrzeć na stosunek zmiany wag do ich wartości - wg nich powinno być ok. 0,1%.

### Wizualizacja warstw
Oczywiście jeśli się da - rzeczy typu "pierwsza warstwa cnn nauczyła się krawędzi [tu obrazki krawędzi], siódma nauczyła się kotów [obrazek kota]"


## Fajne słowa

- Neural Turing Machines - https://arxiv.org/abs/1410.5401
- Attention LSTM/coś -http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/